### Imports

In [76]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from tqdm import tqdm
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import pickle

%run utils.ipynb

15
Char [['m', 's', '>', 'l', 'p'], ['w', 'm', 'n'], ['H', 'n', 'v'], ['w', 'h', 'w'], ['q', 'A', 'd', 'r'], ['E', 'l', 'Y'], ['A', 'l', '<', 'T', 'E', 'A', 'm'], ['>', 'w'], ['A', 'l', 'k', 's', 'w', 'p'], ['>', 'w'], ['A', 'l', 'E', 't', 'q'], ['v', 'm'], ['A', 'f', 't', 'q', 'r'], ['f', 'E', 'j', 'z'], ['E', 'n'], ['k', 'l'], ['*', 'l', 'k'], ['l', 'm'], ['y', 'j', 'z', 'h'], ['A', 'l', 'S', 'w', 'm'], ['>', 'S', 'l', 'A']]
Diac [['a', 'o', 'a', 'a', 'N'], ['a', 'a', 'o'], ['a', 'i', 'a'], ['a', 'u', 'a'], ['a', ' ', 'i', 'N'], ['a', 'a', ' '], [' ', ' ', 'i', 'o', 'a', ' ', 'i'], ['a', 'o'], [' ', 'o', 'i', 'o', 'a', 'i'], ['a', 'o'], [' ', 'o', 'i', 'o', 'i'], ['u', '~a'], [' ', 'o', 'a', 'a', 'a'], ['a', 'a', 'a', 'a'], ['a', 'o'], ['u', '~i'], ['a', 'i', 'a'], ['a', 'o'], ['u', 'o', 'i', 'i'], [' ', ' ', '~a', 'o', 'u'], ['a', 'o', 'F', ' ']]
38
15


### Model building

In [77]:
class RNN(nn.Module):
    def __init__(self, vocab_size, n_classes, embedding_dim = 200, hidden_size = 256,num_layers=3):
        """
        The constructor of our RNN model
        Inputs:
        - vacab_size: the number of unique characters
        - embedding_dim: the embedding dimension
        - n_classes: the number of final classes (diacritics)
        """
        super(RNN, self).__init__()

        # (1) Create the embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # (2) Create an LSTM layer with hidden size = hidden_size and batch_first = True
        self.lstm = nn.LSTM(embedding_dim, hidden_size,num_layers=num_layers, batch_first=True,bidirectional=True)

        # (3) Create a linear layer with number of neorons = n_classes
        self.linear = nn.Linear(hidden_size*2, n_classes)

    def forward(self, sentences):
        """
        This function does the forward pass of our model
        Inputs:
        - sentences: tensor of shape (batch_size, max_length)

        Returns:
        - final_output: tensor of shape (batch_size, max_length, n_classes)
        """

        final_output = None
        
        embeddings = self.embedding(sentences)
        lstm_out, _ = self.lstm(embeddings)
        output = self.linear(lstm_out)
        # final_output = F.softmax(output, dim=1)
        return output

In [78]:
save_path="./models/lstm.pth"

### Train

In [79]:
def train(model, train_dataset, train_labels, batch_size=512, epochs=5, learning_rate=0.001):
    """
    This function implements the training logic
    Inputs:
    - model: the model to be trained
    - train_dataset: the training set
    - batch_size: integer represents the number of examples per step
    - epochs: integer represents the total number of epochs (full training pass)
    - learning_rate: the learning rate to be used by the optimizer
    """

    # (1) create the dataloader of the training set (make the shuffle=True)
    tensor_train_dataset = TensorDataset(train_dataset, train_labels)
    train_dataloader = DataLoader(tensor_train_dataset, batch_size=batch_size, shuffle=True)

    # (2) make the criterion cross entropy loss
    criterion = torch.nn.CrossEntropyLoss()

    # (3) create the optimizer (Adam)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # GPU configuration
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()
    
    best_accuracy = 0.0
    for epoch_num in range(epochs):
        total_acc_train = 0
        total_loss_train = 0
        
        for train_input, train_label in tqdm(train_dataloader):

            # (4) move the train input to the device
            train_label = train_label.to(device)

            # (5) move the train label to the device
            train_input = train_input.to(device)

            # (6) do the forward pass
            output = model(train_input)

            # (7) loss calculation (you need to think in this part how to calculate the loss correctly)
            batch_loss = criterion(output, train_label)

            # (8) append the batch loss to the total_loss_train
            total_loss_train += batch_loss
            
            # (9) calculate the batch accuracy (just add the number of correct predictions)
            acc = (output.argmax(2) == train_label).sum().item()

            total_acc_train += acc

            # (10) zero your gradients
            optimizer.zero_grad()

            # (11) do the backward pass
            batch_loss.backward()

            # (12) update the weights with your optimizer
            optimizer.step()     
        
        # epoch loss
        epoch_loss = total_loss_train / len(train_dataset)

        # (13) calculate the accuracy
        epoch_acc = total_acc_train / (len(train_dataset) * train_dataset[0][0].shape[0])
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {epoch_loss} \
            | Train Accuracy: {epoch_acc}\n')
        if epoch_acc > best_accuracy:
            best_accuracy = epoch_acc
            torch.save(model.state_dict(), save_path)
            print(f'Saved the best model with accuracy: {best_accuracy} to {save_path}')


In [80]:
corpus=  readFile(TRAIN_PATH)
valid_corpus = readFile(VAL_PATH)

In [81]:
X_train = []
y_train = []

max_sequence_length = 0

for sentence in corpus[:10]:
	# Clean each sentence in the corpus
	clean_sentence = run_buckwalter(sentence.strip())
	# Get the char list for each word in the sentence and its corresponding diacritics
	char_list, diacritics_list = extract_labels(clean_sentence)

	X_train.append(char_list)
	y_train.append(diacritics_list)

	# Get the max sequence length and concatenate the embeddings of the words
	for word in char_list:
		max_sequence_length = max(max_sequence_length, len(word))




In [85]:
X_train_padded = [torch.tensor([char_to_index[char] for char in word]) for sentence in X_train for word in sentence ]
X_train_padded = pad_sequence(X_train_padded, batch_first=True)

y_train_padded = [torch.tensor([diacritic_to_index[char] for char in word]) for sentence in y_train for word in sentence ]
print(y_train_padded)
y_train_padded = pad_sequence(y_train_padded, batch_first=True)
print(y_train_padded)

[tensor([2, 1, 5, 5]), tensor([2, 1]), tensor([2, 2, 2]), tensor([ 9,  1,  2, 10,  5]), tensor([2, 2, 5]), tensor([9, 2, 1]), tensor([2, 9, 2]), tensor([ 9,  9, 10,  1,  2,  3, 12]), tensor([9, 1, 5]), tensor([2, 2, 2, 2]), tensor([2, 1, 5, 5]), tensor([3, 2, 1, 8]), tensor([2, 1, 2, 3, 9, 9]), tensor([2, 3, 1, 2, 9, 3]), tensor([2, 1, 3]), tensor([2, 3, 9, 8]), tensor([3, 9, 1, 3, 1, 2, 9, 3]), tensor([5, 5, 9, 2]), tensor([2, 9]), tensor([5, 3, 2]), tensor([5, 5, 9, 5, 5]), tensor([3, 1]), tensor([ 9,  9, 11,  9,  3]), tensor([2, 5, 9, 2, 7]), tensor([2, 3, 1, 2, 9, 3]), tensor([5, 1, 2, 8]), tensor([3, 2, 2, 8]), tensor([ 2,  2, 11]), tensor([ 5, 10,  9,  8]), tensor([9, 1, 5]), tensor([2, 2, 2, 2]), tensor([2, 1, 5]), tensor([9, 1, 3]), tensor([2, 9, 8]), tensor([2, 1]), tensor([3, 3, 1, 8]), tensor([ 2,  2,  2, 10,  5,  5]), tensor([5, 2]), tensor([2, 5, 1, 3]), tensor([ 9,  9, 12, 10,  9,  3]), tensor([2, 3, 1, 2, 9, 3]), tensor([9, 1, 5, 1, 2, 3]), tensor([3, 9]), tensor([2, 3, 

In [83]:
model=RNN(len(unique_characters)+1,len(unique_diacritics),embedding_dim=200,hidden_size=256,num_layers=3)
print(model)


RNN(
  (embedding): Embedding(39, 200)
  (lstm): LSTM(200, 256, num_layers=3, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=15, bias=True)
)


In [84]:
train(model,X_train_padded,y_train_padded,batch_size=512,epochs=5,learning_rate=0.001)

  0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: Expected target size [394, 15], got [394, 9]